In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [4]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 📥 Загрузка
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [7]:
df.head(10)

PassengerId HomePlanet CryoSleep  Cabin    Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P    TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S    TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S    TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S    TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S    TRAPPIST-1e  16.0  False   
5     0005_01      Earth     False  F/0/P  PSO J318.5-22  44.0  False   
6     0006_01      Earth     False  F/2/S    TRAPPIST-1e  26.0  False   
7     0006_02      Earth      True  G/0/S    TRAPPIST-1e  28.0  False   
8     0007_01      Earth     False  F/3/S    TRAPPIST-1e  35.0  False   
9     0008_01     Europa      True  B/1/P    55 Cancri e  14.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck                Name  \
0          0.0        0.0           0.0     0.0     0.0     Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0        Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0       Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0        Solam Susent   
4        303.0       70.0         151.0   565.0     2.0   Willy Santantines   
5          0.0      483.0           0.0   291.0     0.0   Sandie Hinetthews   
6         42.0     1539.0           3.0     0.0     0.0  Billex Jacostaffey   
7          0.0        0.0           0.0     0.0     NaN  Candra Jacostaffey   
8          0.0      785.0          17.0   216.0     0.0       Andona Beston   
9          0.0        0.0           0.0     0.0     0.0      Erraiam Flatic   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  
5         True  
6         True  
7         True  
8         True  
9         True

In [8]:
# 🎯 Цель
df["Transported"] = df["Transported"].astype(int)

In [9]:
# 🧠 Feature Engineering
def enrich(df):# Заметка 1
    df[["Deck", "Num", "Side"]] = df["Cabin"].str.split("/",expand = True) # Разделяем строку в колонке кабина str.split("/", expand=True)
                                                                           # Заметка 2,3,4  
    df["Group"] = df["PassengerId"].apply(lambda x: x.split("_")[0]) # Заметка 5,6,7
    
    df["NameLength"] = df["Name"].fillna("").apply(len)# Заметка 8,9,10
    
    df["TotalSpend"] = df[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].fillna(0).sum(axis=1) # Заметка 11
    
    df["IsZeroSpend"] = (df["TotalSpend"] == 0).astype(int) # Заметка 12
    
    df["CryoZero"] = ((df["CryoSleep"] == True) & (df["TotalSpend"] == 0)).astype(int) # Заметка 13
    
    df["IsAlone"] = df["Group"].map(df["Group"].value_counts()) == 1 # Заметка 14
    
    df["SideIsP"] = (df["Side"] == "P").astype(int) # Заметка 15
    return df

df = enrich(df)
df_test = enrich(df_test)

📌 Итого:

1. 🧠 def enrich(df):
Это функция — мы создаём блок кода, который можно вызывать сколько угодно раз.
Она будет работать с таблицей (DataFrame), которую мы назовём df.
Типа "обработай вот эту таблицу".
2. 👨🏻‍💻split("/") — разбивает строку на части;
3. 👨🏻‍💻expand=True — разворачивает список в колонки;
4. 👨🏻‍💻Без expand=True всё будет в одной колонке (со списками внутри), и дальше с этим сложно работать.
5. 👨🏻‍💻Метод .apply() используется, чтобы вызвать функцию на каждой строке колонки.
означает:
«Пройди по каждому значению в колонке и применяй к нему вот эту функцию».
6. 👨🏻‍💻(lambda) Это анонимная функция, которую мы применяем к каждому значению. Значение x означает 1234_4
7. 👨🏻‍💻x.split('_')[0] делит значение x то есть 1234_4 без разделителя _ и на выходе выдает значение 0 то есть 1234
8. 👨🏻‍💻В колонке Name может быть пропуск (NaN).
9. 👨🏻‍💻fillna("") — заменяем пропуск на пустую строку.
10. 👨🏻‍💻apply(len) — считаем длину имени.
11. Вывели колонки с тратами с помощью fillna(0) дали Nan значением = 0, после чего суммировали их с помощью sum(axis=1), axis = 0 это по вертикали, axis = 1 по горизонтали, то есть мы суммировали всё по горизонтали
12. Вывели колонку "0 трат" и создали булевую проверку == 0 то есть если df"TotalSpend" равен 0 то ставим True, если больше 0 то False, с помощью astype(int) меняем значение True/False на 0 или 1 на выходе получаем 0 либо 1
13. Вывели колонку "Не было в креосне" и с помощь & проверяем два условия одновременно: Был ли в креосне? и при этому ничего не потратил
если оба варианта выполняются то ставится True, если 1 условия или 2 не выполняются значит False так же с помощью astype(int) переводим True/Fasle в 0 или 1
14. Вывели колонку "Один ли в группе?" и с помощью map мы как бы сказали "Пройдись по всей колонке Group и каждому пассажиру подставь, сколько раз его группа встречается" и поставили если == 1 то True Значит ехал один, если False значит ехал с кем-то
15. Вывели колонку "сторна P" и сказали == P то есть если есть в колонке Side "P" то это True и можно закидывать в колонку "сторна P", если False nо нет и с помощью astype(int) вывели True/False на 0/1

In [11]:
df.head(10)

PassengerId HomePlanet CryoSleep  Cabin    Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P    TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S    TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S    TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S    TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S    TRAPPIST-1e  16.0  False   
5     0005_01      Earth     False  F/0/P  PSO J318.5-22  44.0  False   
6     0006_01      Earth     False  F/2/S    TRAPPIST-1e  26.0  False   
7     0006_02      Earth      True  G/0/S    TRAPPIST-1e  28.0  False   
8     0007_01      Earth     False  F/3/S    TRAPPIST-1e  35.0  False   
9     0008_01     Europa      True  B/1/P    55 Cancri e  14.0  False   

   RoomService  FoodCourt  ShoppingMall  ...  Deck  Num Side  Group  \
0          0.0        0.0           0.0  ...     B    0    P   0001   
1        109.0        9.0          25.0  ...     F    0    S   0002   
2         43.0     3576.0           0.0  ...     A    0    S   0003   
3          0.0     1283.0         371.0  ...     A    0    S   0003   
4        303.0       70.0         151.0  ...     F    1    S   0004   
5          0.0      483.0           0.0  ...     F    0    P   0005   
6         42.0     1539.0           3.0  ...     F    2    S   0006   
7          0.0        0.0           0.0  ...     G    0    S   0006   
8          0.0      785.0          17.0  ...     F    3    S   0007   
9          0.0        0.0           0.0  ...     B    1    P   0008   

  NameLength TotalSpend IsZeroSpend CryoZero  IsAlone  SideIsP  
0         15        0.0           1        0     True        1  
1         12      736.0           0        0     True        0  
2         13    10383.0           0        0    False        0  
3         12     5176.0           0        0    False        0  
4         17     1091.0           0        0     True        0  
5         17      774.0           0        0     True        1  
6         18     1584.0           0        0    False        0  
7         18        0.0           1        1    False        0  
8         13     1018.0           0        0     True        0  
9         14        0.0           1        1    False        1  

[10 rows x 24 columns]

In [20]:
# === Удаляем редкие категории ===
for col in ["HomePlanet", "Destination", "Deck", "Side"]: # Заметка 1
    freq = df[col].value_counts() # Заметка 2
    rare = freq[freq < 10].index #Заметка 3
    df[col] = df[col].replace(rare, "Rare") #Заметка 4
    df_test[col] = df_test[col].replace(rare, "Rare") #Заметка 5

📌 Итого:

1. Создаем цикл col в котором будем переберать 4 категории "HomePlanet", "Destination", "Deck", "Side"
2. Создаем переменную freq даем ей датафрейм с циклом col и считаем с помощью value_counts сколько раз встречается категория
3. Создаем переменную rare даем ей freq внутри которой freq меньше 10 то есть если категория встречалась 10 раз или меньше то ее закидываем в новую категорию rare и .index делает так # → Index(['Europa', 'Namek'], dtype='object') еслі без него так # Europa     8 # Namek      2
4. Теперь в датафрейме ми меняем все редкие значения по типу 'Europa', 'Namek' на Rare
5. То же самое делаем и для датафрейма тестовой версии

🧠 Зачем это нужно:

Категориальные признаки с редкими значениями → плохи для модели
Они не несут полезной статистики, могут вызвать переобучение
Замена на "Rare" — простой и эффективный приём

In [23]:
# 🧹 Подготовка
drop_cols = ["Transported", "PassengerId", "Name", "Cabin"]
X = df.drop(columns=drop_cols)
y = df["Transported"]
X_test = df_test.drop(columns=["PassengerId", "Name", "Cabin"])

In [29]:
df.head(10)

PassengerId HomePlanet CryoSleep  Cabin    Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P    TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S    TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S    TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S    TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S    TRAPPIST-1e  16.0  False   
5     0005_01      Earth     False  F/0/P  PSO J318.5-22  44.0  False   
6     0006_01      Earth     False  F/2/S    TRAPPIST-1e  26.0  False   
7     0006_02      Earth      True  G/0/S    TRAPPIST-1e  28.0  False   
8     0007_01      Earth     False  F/3/S    TRAPPIST-1e  35.0  False   
9     0008_01     Europa      True  B/1/P    55 Cancri e  14.0  False   

   RoomService  FoodCourt  ShoppingMall  ...  Deck  Num Side  Group  \
0          0.0        0.0           0.0  ...     B    0    P   0001   
1        109.0        9.0          25.0  ...     F    0    S   0002   
2         43.0     3576.0           0.0  ...     A    0    S   0003   
3          0.0     1283.0         371.0  ...     A    0    S   0003   
4        303.0       70.0         151.0  ...     F    1    S   0004   
5          0.0      483.0           0.0  ...     F    0    P   0005   
6         42.0     1539.0           3.0  ...     F    2    S   0006   
7          0.0        0.0           0.0  ...     G    0    S   0006   
8          0.0      785.0          17.0  ...     F    3    S   0007   
9          0.0        0.0           0.0  ...     B    1    P   0008   

  NameLength TotalSpend IsZeroSpend CryoZero  IsAlone  SideIsP  
0         15        0.0           1        0     True        1  
1         12      736.0           0        0     True        0  
2         13    10383.0           0        0    False        0  
3         12     5176.0           0        0    False        0  
4         17     1091.0           0        0     True        0  
5         17      774.0           0        0     True        1  
6         18     1584.0           0        0    False        0  
7         18        0.0           1        1    False        0  
8         13     1018.0           0        0     True        0  
9         14        0.0           1        1    False        1  

[10 rows x 24 columns]

In [25]:
# 📌 Категориальные признаки
cat_features = X.select_dtypes(include="object").columns.tolist()
for col in cat_features:
    X[col] = X[col].astype(str).fillna("missing")
    X_test[col] = X_test[col].astype(str).fillna("missing")

In [27]:
# ✂️ Разделение
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [28]:
# 🐱 Модель CatBoost
model = CatBoostClassifier(
    auto_class_weights='Balanced',
    iterations=1000,
    early_stopping_rounds=50,
    learning_rate=0.02,
    depth=7,
    l2_leaf_reg=5,
    cat_features=cat_features,
    verbose=100,
    random_state=42
)

In [30]:
# 🏋 Обучение
model.fit(X_train, y_train)

0:	learn: 0.6837547	total: 75.6ms	remaining: 1m 15s
100:	learn: 0.4318498	total: 838ms	remaining: 7.46s
200:	learn: 0.4017310	total: 1.55s	remaining: 6.17s
300:	learn: 0.3871277	total: 2.26s	remaining: 5.25s
400:	learn: 0.3761405	total: 3.03s	remaining: 4.53s
500:	learn: 0.3652229	total: 3.83s	remaining: 3.82s
600:	learn: 0.3547283	total: 4.68s	remaining: 3.11s
700:	learn: 0.3448427	total: 5.5s	remaining: 2.35s
800:	learn: 0.3356767	total: 6.38s	remaining: 1.58s
900:	learn: 0.3262494	total: 7.22s	remaining: 794ms
999:	learn: 0.3184973	total: 8.05s	remaining: 0us


In [31]:
# 🧪 Оценка
y_pred = model.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
print(f"✅ CatBoost accuracy: {acc:.4f}")

✅ CatBoost accuracy: 0.7987


In [32]:
# 📤 Предсказание
y_test_pred = model.predict(X_test)
submission["Transported"] = y_test_pred.astype(bool)
submission.to_csv("submission_catboost1.csv", index=False)
print("📁 Файл submission_catboost1.csv сохранён")

📁 Файл submission_catboost1.csv сохранён


In [33]:
from sklearn.preprocessing import LabelEncoder

# Копия датафреймов, чтобы не испортить CatBoost
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()

for col in X_train_enc.select_dtypes(include="object").columns:
    le = LabelEncoder()
    all_vals = pd.concat([X_train_enc[col], X_test_enc[col]], axis=0).astype(str)
    le.fit(all_vals)

    X_train_enc[col] = le.transform(X_train_enc[col].astype(str))
    X_test_enc[col] = le.transform(X_test_enc[col].astype(str))

In [34]:
# === Обучение CatBoost ===
cb_model = CatBoostClassifier(
    iterations=2000, learning_rate=0.03, depth=6,
    cat_features=cat_features, early_stopping_rounds=50,
    verbose=0, random_state=42
)
cb_model.fit(X_train, y_train, eval_set=(X_valid, y_valid))
cb_preds = cb_model.predict_proba(X_test)[:, 1]

# === Обучение LightGBM ===
lgb_model = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
lgb_model.fit(X_train_enc, y_train)
lgb_preds = lgb_model.predict_proba(X_test_enc)[:, 1]

# === Обучение XGBoost ===
xgb_model = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6,
                          eval_metric="logloss", random_state=42)
xgb_model.fit(X_train_enc, y_train)
xgb_preds = xgb_model.predict_proba(X_test_enc)[:, 1]

# === Взвешенный блендинг + смещённый порог ===
blended_probs = (0.5 * cb_preds + 0.3 * lgb_preds + 0.2 * xgb_preds)
submission["Transported"] = (blended_probs > 0.47).astype(bool)
submission.to_csv("submission_blend3_weighted_047.csv", index=False)
print("✅ submission_blend3_weighted_047.csv сохранён")

[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000677 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2166
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i